In [1]:

import sys
!{sys.executable} -m pip install -r requirements.txt

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
     |████████████████████████████████| 296kB 886kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/b7/17/a83f31f973d0fa8724db4106a187c38715727e5edfe947174abff482d7ff/mongoengine-0.10.7.tar.gz
  Created wheel for mongoengine: filename=mongoengine-0.10.7-cp27-none-any.whl size=92374 sha256=83ab651600efe75a11ffe46afd2af725041f41aef66319791dca526152d0ad93
  Stored in directory: /Users/mac/Library/Caches/pip/wheels/0d/6f/fc/d9b4b4b23c5324145dfc4b3e8e464114876aafc427a7bcfb71
Successfully built mongoengine


In [3]:
from tqdm import tqdm
import pandas as pd
import pprint
import project_helper

import lxml

from pymongo import MongoClient



In [4]:
cik_lookup = {
    'AMZN': '0001018724'}
#     'BMY': '0000014272',   
#     'CNP': '0001130310',
#     'CVX': '0000093410',
#     'FL': '0000850209',
#     'FRT': '0000034903',
#     'HON': '0000773840'}

In [6]:
client = MongoClient('localhost', 27017)
client = MongoClient('mongodb://localhost:27017')
db = client.pymongo_test
db = client['pymongo_test']


In [7]:
sec_api = project_helper.SecAPI()


In [8]:
%psource project_helper


In [77]:
from bs4 import BeautifulSoup
import re

def get_sec_data(cik, doc_type='',dateb='', owner='include', start=0, count=100,search_text=''):
    newest_pricing_data = pd.to_datetime('2018-01-01')

    
    
    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
        '&CIK={}&type={}&dateb={}&owner={}&count={}&search_text={}' \
        .format(cik, doc_type, dateb, owner , start ,count , search_text)
    
#     print ( rss_url)
    
    
    sec_data = sec_api.get(rss_url)
    
    
#     print (sec_data)
    
    soup = BeautifulSoup(sec_data, 'html.parser')
    
    print(soup.find_all(text=re.compile("(.*?)")))
    

    
#     for strong_tag in soup.find_all('td'):
#         print(strong_tag.text)
    
    
#     x = soup.find_all('td')
    
#     y = soup.getText()
#     print(type(y))
    
#     z = re.match("/(\d{4}-\d{2}-\d{1,2}).*/", y)
    
#     print(z)



    
    
#     print (soup.find_all(re.compile('/(\d{4}-\d{2}-\d{1,2}).*/')))


    
    
#     feed = BeautifulSoup(sec_data.encode('ascii'), 'xml').feed
    
#     print(feed)
    
    
#     entries = [
#         (
#             entry.content.find('filing-href').getText(),
#             entry.content.find('filing-type').getText(),
#             entry.content.find('filing-date').getText())
#         for entry in feed.find_all('entry', recursive=False)
#         if pd.to_datetime(entry.content.find('filing-date').getText()) <= newest_pricing_data]

#     return entries
    
# #     print(feed.find_all('filing-date'))
    

    
#     entries = [
#         (
# #             entry.content.find('filing-href').getText(),entry.content.find('filing-type').getText(),
#             entry.content.find('filing-date').getText())
        
#         for entry in feed.find_all('entry', recursive=False)
        
#         if pd.to_datetime(entry.content.find('filing-date').getText()) <= newest_pricing_data]
   

#     return entries

In [78]:
example_ticker = 'AMZN'
sec_data = {}


for ticker, cik in cik_lookup.items():
    sec_data[ticker] = get_sec_data(cik)

pprint.pprint(sec_data[example_ticker][:5])

[u'HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"', u'\n', u'\n', u'\n', u'EDGAR Search Results', u'\n', u'\n', u'\n', u'\n', u'\n', u'\n', u'\n', u' SEC Web Analytics - For information please visit: https://www.sec.gov/privacy.htm#collectedinfo ', u'\n', u'\n', u"(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':\nnew Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],\nj=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=\n'//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);\n})(window,document,'script','dataLayer','GTM-TD3BKV');", u'\n', u' End SEC Web Analytics ', u'\n', u' BEGIN BANNER ', u'\n', u'\n', u'Home', u' | ', u'Latest Filings', u' | ', u'Previous Page', u'\n', u'\n', u'\n', u'\n', u'\n', u' ', u'EDGAR Search Results', u'\n', u'\n', u' END BANNER ', u'\n', u' BEGIN BREADCRUMBS ', u'\n', u'\n', u'\n', u'SEC Home', u' \xbb', u'\n', u'Search the Next-Generation EDGA

TypeError: 'NoneType' object has no attribute '__getitem__'

In [36]:
sec_data

{'AMZN': None}

In [47]:
raw_fillings_by_ticker = {}

for ticker, data in sec_data.items():
    raw_fillings_by_ticker[ticker] = {}
    for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
        if (file_type == '10-K'):
            file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            
            
            raw_fillings_by_ticker[ticker][file_date] = sec_api.get(file_url)


print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker[example_ticker].values()))[:1000]))

Example Document:

-----BEGIN PRIVACY-ENHANCED MESSAGE-----
Proc-Type: 2001,MIC-CLEAR
Originator-Name: webmaster@www.sec.gov
Originator-Key-Asymmetric:
 MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen
 TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB
MIC-Info: RSA-MD5,RSA,
 JheN9rrH0LADcPLNpdgic2Y+wa34llXqe750TowVzUFLd6IuuNQne2H0axAosYFz
 1Lmg9dJfEMxBnv2E6NCBrw==

<SEC-DOCUMENT>0001193125-05-047032.txt : 20050311
<SEC-HEADER>0001193125-05-047032.hdr.sgml : 20050311
<ACCEPTANCE-DATETIME>20050311062027
ACCESSION NUMBER:		0001193125-05-047032
CONFORMED SUBMISSION TYPE:	10-K
PUBLIC DOCUMENT COUNT:		9
CONFORMED PERIOD OF REPORT:	20041231
FILED AS OF DATE:		20050311
DATE AS OF CHANGE:		20050311

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			AMAZON COM INC
		CENTRAL INDEX KEY:			0001018724
		STANDARD INDUSTRIAL CLASSIFICATION:	RETAIL-CATALOG & MAIL-ORDER HOUSES [5961]
		IRS NUMBER:				911646860
		STATE OF INCORPORATION:			DE
		FISCAL YEAR END:			1231

	FILING VA